# Exercise - Multi step Workflow - STARTER

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from rich.pretty import Pretty

import os

## 1. Instantiate Chat Model with your API Key

To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key="voc-",
)
```

In [2]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key = os.getenv("OPEN_AI_API_KEY"),
)

## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [3]:
logs = []

In [4]:
parser = StrOutputParser()

In [5]:
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [6]:
idea_prompt = PromptTemplate(
    template=(
        "Come up with a business idea for {industry}"
    )
)
Pretty(idea_prompt)

PromptTemplate(
    input_variables=['industry'],
    input_types={},
    partial_variables={},
    template='Come up with a business idea for {industry}'
)

In [7]:
# Create your idea_chain: idea_prompt -> llm -> parse_and_log_output_chain
idea_chain = idea_prompt | llm | parse_and_log_output_chain
idea_chain

PromptTemplate(input_variables=['industry'], input_types={}, partial_variables={}, template='Come up with a business idea for {industry}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f445b954410>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f445b3b1d90>, root_client=<openai.OpenAI object at 0x7f445b9c6f00>, root_async_client=<openai.AsyncOpenAI object at 0x7f445b9544d0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| {
    output: StrOutputParser(),
    log: RunnableLambda(lambda x: logs.append(x))
  }

In [8]:
idea_chain.get_graph().print_ascii()

              +-------------+            
              | PromptInput |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *                  
       +---------------------------+     
       | Parallel<output,log>Input |     
       +---------------------------+     
               ***         ***           
              *               *          
            **                 ** 

In [9]:
# Test your idea_chain invoking it by passing an industy like "agro" to it
idea_result = idea_chain.invoke("argo")
Pretty(idea_result)

{
    'output': '**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven 
agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI 
(Artificial Intelligence), and data analytics to optimize farming practices, increase crop yields, and promote 
sustainable agriculture.\n\n**Key Components:**\n\n1. **IoT-Enabled Smart Sensors:**\n   - Develop and deploy a 
range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time. These 
sensors can provide farmers with actionable insights to optimize irrigation and fertilization.\n\n2. **AI-Powered 
Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from the sensors and uses AI 
algorithms to analyze trends, predict crop performance, and recommend best practices tailored to specific crops and
local conditions.\n\n3. **Mobile App for Farmers:**\n   - Develop a user-friendly mobile application that allows 
farmers to access real-time data, receive alerts, and get personalized recommendations. The app can also include 
features for tracking crop health, weather forecasts, and market prices.\n\n4. **Sustainable Farming Practices:**\n
- Promote sustainable agriculture by providing resources and tools for organic farming, crop rotation, and 
integrated pest management. Partner with agricultural experts to offer training and workshops.\n\n5. **Subscription
Model:**\n   - Offer a subscription-based model for farmers to access the IoT devices, analytics platform, and 
mobile app. This model can include different tiers based on the size of the farm and the level of service 
required.\n\n6. **Partnerships with Agricultural Institutions:**\n   - Collaborate with universities, research 
institutions, and agricultural organizations to stay at the forefront of agricultural research and innovation. This
can also help in gaining credibility and expanding the customer base.\n\n7. **Community Engagement:**\n   - Build a
community platform where farmers can share experiences, best practices, and success stories. This can foster 
collaboration and knowledge sharing among farmers.\n\n**Target Market:**\n- Small to medium-sized farmers looking 
to enhance productivity and sustainability.\n- Agricultural cooperatives and organizations focused on improving 
farming practices.\n- Government agencies and NGOs working in agricultural development.\n\n**Revenue Streams:**\n- 
Subscription fees for the analytics platform and mobile app.\n- Sales of IoT devices and sensors.\n- Consulting 
services for sustainable farming practices.\n- Workshops and training sessions.\n\n**Impact:**\nArgo aims to 
empower farmers with the tools and knowledge they need to make informed decisions, ultimately leading to increased 
productivity, reduced environmental impact, and improved food security. By harnessing technology, Argo can help 
transform traditional farming into a more efficient and sustainable practice.',
    'log': None
}

In [10]:
idea_result["output"]

'**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices, increase crop yields, and promote sustainable agriculture.\n\n**Key Components:**\n\n1. **IoT-Enabled Smart Sensors:**\n   - Develop and deploy a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time. These sensors can provide farmers with actionable insights to optimize irrigation and fertilization.\n\n2. **AI-Powered Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from the sensors and uses AI algorithms to analyze trends, predict crop performance, and recommend best practices tailored to specific crops and local conditions.\n\n3. **Mobile App for Farmers:**\n   - Develop a user-friendly mobile application that allows farmers to ac

In [11]:
logs

[AIMessage(content='**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices, increase crop yields, and promote sustainable agriculture.\n\n**Key Components:**\n\n1. **IoT-Enabled Smart Sensors:**\n   - Develop and deploy a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time. These sensors can provide farmers with actionable insights to optimize irrigation and fertilization.\n\n2. **AI-Powered Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from the sensors and uses AI algorithms to analyze trends, predict crop performance, and recommend best practices tailored to specific crops and local conditions.\n\n3. **Mobile App for Farmers:**\n   - Develop a user-friendly mobile application that a

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [12]:
analysis_prompt = PromptTemplate(
    template=(
        "Analyze the following business idea:\n" \
        "{idea}\n" \
        "List strengths and weaknesses as bullet points."
    )
)

In [13]:
analysis_chain = (
    analysis_prompt | llm | parse_and_log_output_chain
)

In [14]:
# Test your analysis_chain invoking it by passing idea_result["output"] to it
analysis_result = analysis_chain.invoke(idea_result["output"])

In [15]:
Pretty(analysis_result["output"])

'### Strengths:\n- **Innovative Technology Integration:** Utilizes cutting-edge technologies (IoT, AI, data 
analytics) to provide real-time insights and optimize farming practices.\n- **Focus on Sustainability:** Promotes 
sustainable agricultural practices, which can attract environmentally conscious consumers and farmers.\n- 
**User-Friendly Mobile App:** Offers a convenient platform for farmers to access critical information and 
recommendations, enhancing user engagement.\n- **Subscription Model:** Provides a steady revenue stream and allows 
for scalable growth based on customer needs and farm sizes.\n- **Partnership Opportunities:** Collaborations with 
agricultural institutions can enhance credibility, foster innovation, and expand the customer base.\n- **Community 
Engagement:** Encourages knowledge sharing and collaboration among farmers, which can lead to improved practices 
and customer loyalty.\n- **Diverse Revenue Streams:** Multiple income sources (subscriptions, device sales, 
consulting, workshops) can mitigate financial risks.\n\n### Weaknesses:\n- **High Initial Investment:** Development
and deployment of IoT devices and AI platforms may require significant upfront capital and resources.\n- 
**Technology Adoption Barriers:** Some farmers may be resistant to adopting new technologies due to lack of 
familiarity or perceived complexity.\n- **Dependence on Internet Connectivity:** The effectiveness of IoT devices 
and the analytics platform may be limited in areas with poor internet access.\n- **Market Competition:** The 
agricultural tech space is becoming increasingly crowded, with many competitors offering similar solutions.\n- 
**Ongoing Maintenance Costs:** Continuous updates and maintenance of software and hardware may incur additional 
costs and require dedicated resources.\n- **Scalability Challenges:** Tailoring solutions for diverse crops and 
local conditions may complicate scaling efforts across different regions.\n- **Regulatory Compliance:** Navigating 
agricultural regulations and data privacy laws can be complex and may pose challenges for the business.'

In [16]:
logs

[AIMessage(content='**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices, increase crop yields, and promote sustainable agriculture.\n\n**Key Components:**\n\n1. **IoT-Enabled Smart Sensors:**\n   - Develop and deploy a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time. These sensors can provide farmers with actionable insights to optimize irrigation and fertilization.\n\n2. **AI-Powered Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from the sensors and uses AI algorithms to analyze trends, predict crop performance, and recommend best practices tailored to specific crops and local conditions.\n\n3. **Mobile App for Farmers:**\n   - Develop a user-friendly mobile application that a

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [25]:
# Your prompt Template
report_prompt = PromptTemplate(
    template=(
        "You are an analyst having to summarize this analysis of a business idea into a report." \
        "Analysis: {output}" 
    )
)

In [26]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [27]:
report_chain = (
    report_prompt | llm.with_structured_output(AnalysisReport)
)

In [28]:
# Test your report_chain invoking it by passing analysis_result["output"] to it
report_result = report_chain.invoke(analysis_result["output"])

In [29]:
Pretty(report_result)

AnalysisReport(
    strengths=[
        'Innovative Technology Integration: Utilizes cutting-edge technologies (IoT, AI, data analytics) to provide
real-time insights and optimize farming practices.',
        'Focus on Sustainability: Promotes sustainable agricultural practices, which can attract environmentally 
conscious consumers and farmers.',
        'User-Friendly Mobile App: Offers a convenient platform for farmers to access critical information and 
recommendations, enhancing user engagement.',
        'Subscription Model: Provides a steady revenue stream and allows for scalable growth based on customer 
needs and farm sizes.',
        'Partnership Opportunities: Collaborations with agricultural institutions can enhance credibility, foster 
innovation, and expand the customer base.',
        'Community Engagement: Encourages knowledge sharing and collaboration among farmers, which can lead to 
improved practices and customer loyalty.',
        'Diverse Revenue Streams: Multiple income sources (subscriptions, device sales, consulting, workshops) can 
mitigate financial risks.'
    ],
    weaknesses=[
        'High Initial Investment: Development and deployment of IoT devices and AI platforms may require 
significant upfront capital and resources.',
        'Technology Adoption Barriers: Some farmers may be resistant to adopting new technologies due to lack of 
familiarity or perceived complexity.',
        'Dependence on Internet Connectivity: The effectiveness of IoT devices and the analytics platform may be 
limited in areas with poor internet access.',
        'Market Competition: The agricultural tech space is becoming increasingly crowded, with many competitors 
offering similar solutions.',
        'Ongoing Maintenance Costs: Continuous updates and maintenance of software and hardware may incur 
additional costs and require dedicated resources.',
        'Scalability Challenges: Tailoring solutions for diverse crops and local conditions may complicate scaling 
efforts across different regions.',
        'Regulatory Compliance: Navigating agricultural regulations and data privacy laws can be complex and may 
pose challenges for the business.'
    ]
)

## 6. End to End Chain

In [30]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [31]:
e2e_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [32]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [37]:
Pretty(e2e_result)

AnalysisReport(
    strengths=[
        'Innovative Technology: Utilizes advanced technologies like IoT and AI to optimize farming processes, 
enhancing efficiency and crop yield.',
        'Sustainability Focus: Emphasizes sustainable practices, including renewable energy sources and waste 
recycling, appealing to environmentally conscious consumers.',
        'Modular Design: The flexibility of modular units allows for easy installation in various urban settings, 
maximizing space utilization.',
        'Community Engagement: Involves local communities through workshops and partnerships, fostering a sense of 
connection and support for local agriculture.',
        'Subscription Model: Offers a convenient delivery service for fresh produce, catering to the growing demand
for home delivery and personalized options.',
        'Market Demand: Targets a growing market of health-conscious consumers and urban dwellers seeking fresh, 
locally-grown produce.',
        'Research Collaboration: Engages with academic institutions for continuous improvement and innovation in 
farming techniques.'
    ],
    weaknesses=[
        'High Initial Investment: The setup costs for technology, infrastructure, and research may be significant, 
posing a barrier to entry.',
        'Technical Complexity: Reliance on advanced technology may lead to challenges in maintenance and require 
skilled personnel for operation.',
        'Market Competition: The vertical farming industry is becoming increasingly competitive, with many players 
entering the market.',
        'Regulatory Challenges: Urban farming may face zoning laws and regulations that could complicate 
installation and operation.',
        'Consumer Education: There may be a need to educate consumers about the benefits of vertical farming and 
hydroponics/aeroponics to gain market acceptance.',
        'Dependence on Technology: Over-reliance on technology could lead to vulnerabilities, such as system 
failures or cyber threats.',
        'Seasonal Variability: While vertical farming can mitigate some seasonal challenges, certain crops may 
still be affected by environmental factors, impacting yield consistency.'
    ]
)

In [38]:
Pretty(e2e_result.strengths)

[
    'Innovative Technology: Utilizes advanced technologies like IoT and AI to optimize farming processes, enhancing
efficiency and crop yield.',
    'Sustainability Focus: Emphasizes sustainable practices, including renewable energy sources and waste 
recycling, appealing to environmentally conscious consumers.',
    'Modular Design: The flexibility of modular units allows for easy installation in various urban settings, 
maximizing space utilization.',
    'Community Engagement: Involves local communities through workshops and partnerships, fostering a sense of 
connection and support for local agriculture.',
    'Subscription Model: Offers a convenient delivery service for fresh produce, catering to the growing demand for
home delivery and personalized options.',
    'Market Demand: Targets a growing market of health-conscious consumers and urban dwellers seeking fresh, 
locally-grown produce.',
    'Research Collaboration: Engages with academic institutions for continuous improvement and innovation in 
farming techniques.'
]

In [39]:
Pretty(e2e_result.weaknesses)

[
    'High Initial Investment: The setup costs for technology, infrastructure, and research may be significant, 
posing a barrier to entry.',
    'Technical Complexity: Reliance on advanced technology may lead to challenges in maintenance and require 
skilled personnel for operation.',
    'Market Competition: The vertical farming industry is becoming increasingly competitive, with many players 
entering the market.',
    'Regulatory Challenges: Urban farming may face zoning laws and regulations that could complicate installation 
and operation.',
    'Consumer Education: There may be a need to educate consumers about the benefits of vertical farming and 
hydroponics/aeroponics to gain market acceptance.',
    'Dependence on Technology: Over-reliance on technology could lead to vulnerabilities, such as system failures 
or cyber threats.',
    'Seasonal Variability: While vertical farming can mitigate some seasonal challenges, certain crops may still be
affected by environmental factors, impacting yield consistency.'
]

In [41]:
logs

[AIMessage(content='**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices, increase crop yields, and promote sustainable agriculture.\n\n**Key Components:**\n\n1. **IoT-Enabled Smart Sensors:**\n   - Develop and deploy a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time. These sensors can provide farmers with actionable insights to optimize irrigation and fertilization.\n\n2. **AI-Powered Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from the sensors and uses AI algorithms to analyze trends, predict crop performance, and recommend best practices tailored to specific crops and local conditions.\n\n3. **Mobile App for Farmers:**\n   - Develop a user-friendly mobile application that a

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity